# Catchment Boundaries

Notebook to create the files `CAMELS_DE_catchment_boundaries.geopackage` and `CAMELS_DE_catchment_boundaries.shp`.  
As well as `CAMELS_DE_gauging_stations.geopackage` and `CAMELS_DE_gauging_stations.shp`.  

We always use `EPSG:3035` as the coordinate reference system.

In [1]:
from glob import glob
import os
import pandas as pd
import geopandas as gpd

In [2]:
# get camels_ids from hydromet timeseries
camels_ids = [camels_id.split("_")[-1].split(".csv")[0] for camels_id in glob("../output_data/camels_de/timeseries/*.csv")]

# sort camels_ids
camels_ids = sorted(camels_ids)

print(f"Total number of stations in CAMELS-DE v1: {len(camels_ids)}")

Total number of stations in CAMELS-DE v1: 5


## Save catchment boundaries

Get all the catchment boundaries from the CAMELS-DE dataset and save them in a geopackage and shapefile.

In [3]:
CATCHMENT_PATH = "../../../catchments_harz"

# Mapping catchment IDs to shapefiles
id_mapping = {
    '1': dict(shapefile = "innerste_reservoir_catchment.shp", catchment_id = 1),
    '2': dict(shapefile = "oker_reservoir_catchment.shp", catchment_id = 2),
    '3': dict(shapefile = "ecker_reservoir_catchment.shp", catchment_id = 3),
    '4': dict(shapefile = "soese_reservoir_catchment.shp", catchment_id = 4),
    '5': dict(shapefile = "grane_reservoir_catchment.shp", catchment_id = 5),
}

catchments = []


for camels_id in camels_ids:
    
    # initialize catchment
    catchment_info = id_mapping[camels_id]
    shapefile_path = os.path.join(CATCHMENT_PATH, catchment_info["shapefile"])
    catchment = gpd.read_file(shapefile_path)

    # from epsg:4326 to epsg:3035
    catchment = catchment.to_crs(epsg=3035)

    # add catchment to catchments
    catchments.append(catchment)

# concatenate all catchments into a single GeoDataFrame
catchments = gpd.pd.concat(catchments, ignore_index=True)
catchments.crs = "EPSG:3035"

# rename columns
catchments = catchments.rename(columns={"id": "gauge_id", "name": "gauge_name"})

# drop columns
# catchments = catchments.drop(columns=["result", "area_calc", "area_reported"])

# make directory if not exists
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries", exist_ok=True)
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments", exist_ok=True)

# save catchments
catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/CAMELS_DE_catchments.shp")
catchments.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/catchments/CAMELS_DE_catchments.gpkg", driver="GPKG")

## Save gauging station locations

In [ ]:
# read station locations from topographic attributes
df = pd.read_csv("../output_data/camels_de/CAMELS_DE_topographic_attributes.csv")

# convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["gauge_easting"], df["gauge_northing"]))
gdf.crs = "EPSG:3035"

# only keep columns gauge_id, gauge_name, geometry
gdf = gdf[["gauge_id", "gauge_name", "geometry"]]

# make directory if not exists
os.makedirs("../output_data/camels_de/CAMELS_DE_catchment_boundaries/gauging_stations", exist_ok=True)

# save station locations
gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/gauging_stations/CAMELS_DE_gauging_stations.shp")
gdf.to_file("../output_data/camels_de/CAMELS_DE_catchment_boundaries/gauging_stations/CAMELS_DE_gauging_stations.gpkg", driver="GPKG")